In [1]:
import serial
import threading
import time

accelerometer = []
gyroscope = []
# Define the timeout duration
SERIAL_TIMEOUT = 4  # in seconds

# Function to handle data reception
def receive_data(ser):
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line:
            # print(line)  # Print the received data
            data_str = line
            # Split the data string into two parts using "||" as the separator
            data_parts = data_str.split(" || ")
            # len(data_parts)
            # Iterate over the data parts and parse each set of values
            # for data_part in data_parts:
            values = data_parts[0].split(",")  # Split each set of values by ","
            # print(len(values))
            values = [ (val[0]=='-')*(float(val[1:])*(-1)) + (val[0]!='-')*(float(val)) for val in values]  # Convert values to integers
            accelerometer.append(values)  # Append the first set of values to accelerometer

            values = data_parts[1].split(",")  # Split each set of values by ","
            values = [ (val[0]=='-')*(float(val[1:])*(-1)) + (val[0]!='-')*(float(val)) for val in values]  # Convert values to integers
            gyroscope.append(values)  # Append the second set of values to gyroscope
        else:
            # Reset the timer if data is received
            # print(len(gyroscope))
            global last_received
            last_received = time.time()

# Function to close the serial port after a timeout
def check_idle(ser):
    while True:
        time.sleep(1)  # Check every second
        if time.time() - last_received > SERIAL_TIMEOUT:
            print("Closing serial port due to inactivity.")
            final_data = accelerometer + gyroscope
            ser.close()
            break

# Main function
def main():
    ser = serial.Serial('COM7', 9600)  # Adjust the port name and baud rate
    ser.timeout = SERIAL_TIMEOUT  # Set the serial timeout

    global last_received
    last_received = time.time()  # Initialize the last received time

    # Start the data reception thread
    data_thread = threading.Thread(target=receive_data, args=(ser,), daemon=True)
    data_thread.start()

    # Start the timer thread to check for inactivity
    timer_thread = threading.Thread(target=check_idle, args=(ser,), daemon=True)
    timer_thread.start()

    # Keep the main thread alive
    # while True:
    #     pass

if __name__ == "__main__":
    main()


Exception in thread Thread-5 (receive_data):
Traceback (most recent call last):
  File "c:\Users\Parth Magdum\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Parth Magdum\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Parth Magdum\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Parth Magdum\AppData\Local\Temp\ipykernel_18848\1941039422.py", line 13, in receive_data
  File "c:\Users\Parth Magdum\AppData\Local\Programs\Python\Python311\Lib\site-packages\serial\serialwin32.py", line 285, in read
    ctypes.byref(self._overlapped_read))
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: byref() argument must be a ctypes instance, not 'NoneType'


Closing serial port due to inactivity.


In [2]:
len(accelerometer)

46

In [3]:
import csv
import os
import numpy as np

filename = 'letter_data.csv'
label = 'J' #give label for which you are writing here
file_exists = os.path.exists(filename)

with open(filename, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow([label, accelerometer, gyroscope])

print("Data exported to", filename)


Data exported to letter_data.csv


In [4]:
import pandas as pd
import ast

df = pd.read_csv('letter_data.csv')
df["Accelerometer"] = df["Accelerometer"].apply(lambda x: ast.literal_eval(x)).tolist()
df["Gyroscope"] = df["Gyroscope"].apply(lambda x: ast.literal_eval(x)).tolist()
n = len(df)
sum(df['Label']=='J')

30

In [5]:
df.tail()

,Label,Accelerometer,Gyroscope
887,J,"[[-0.34, -0.36, 0.42], [-0.76, -0.04, 1.19], [...","[[79.28, -403.56, 88.93], [69.64, -461.61, 123..."
888,J,"[[0.03, 0.1, 0.18], [-0.52, -0.57, 0.51], [-1....","[[244.75, -319.89, 97.35], [252.14, -502.56, 1..."
889,J,"[[-0.02, 0.39, 0.28], [-0.52, -0.05, 0.6], [-0...","[[253.11, -327.7, 105.47], [115.6, -488.71, 12..."
890,J,"[[-0.15, 0.38, 0.53], [-0.4, 0.08, 0.56], [-0....","[[48.16, -312.99, 69.58], [131.9, -466.86, 133..."
891,J,"[[-0.19, -0.04, 0.4], [-0.71, -0.07, 1.22], [-...","[[152.34, -453.0, 108.58], [103.7, -381.16, 75..."


In [205]:
df_safe = df